In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('recommender').getOrCreate()

In [0]:
from pyspark.ml.recommendation import ALS

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
df = spark.read.csv('/FileStore/tables/movielens_ratings.csv',inferSchema=True,header=True)

In [0]:
df.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [0]:
df.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [0]:
train_df,test_df = df.randomSplit([0.8,0.2])

In [0]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [0]:
als_model = als.fit(train_df)

In [0]:
als_predictions = als_model.transform(test_df)

In [0]:
als_predictions.show()

+-------+------+------+-----------+
movieId|rating|userId| prediction|
+-------+------+------+-----------+
 31| 3.0| 7| 0.6379115|
 31| 1.0| 19| 3.4333267|
 85| 3.0| 1| 0.6818663|
 85| 1.0| 4| 3.3226976|
 85| 1.0| 12| 0.27834564|
 85| 1.0| 13| 1.5912156|
 85| 1.0| 28| 1.8122019|
 65| 5.0| 23| 2.039901|
 53| 1.0| 7| 2.5007927|
 53| 3.0| 13| 3.2406323|
 53| 3.0| 14| 5.605443|
 53| 2.0| 19| 2.970887|
 53| 1.0| 23| 0.64325595|
 78| 1.0| 8| 1.0117165|
 34| 1.0| 4| 0.27123496|
 34| 1.0| 14| 1.1919546|
 34| 1.0| 16| 3.3413286|
 81| 1.0| 16|-0.98031354|
 76| 3.0| 7| 3.8629518|
 26| 1.0| 3| 2.2424262|
+-------+------+------+-----------+
only showing top 20 rows

In [0]:
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='rating',metricName='rmse')

In [0]:
rmse = evaluator.evaluate(als_predictions)

In [0]:
print('RMSE')
print(rmse)

RMSE
1.7642092316134323

In [0]:
user1 = test_df.filter(test_df['userId']==11).select(['movieId','userId'])

In [0]:
user1.show()

+-------+------+
movieId|userId|
+-------+------+
 10| 11|
 11| 11|
 12| 11|
 13| 11|
 27| 11|
 30| 11|
 45| 11|
 64| 11|
 69| 11|
 94| 11|
+-------+------+

In [0]:
recommendations = als_model.transform(user1)

In [0]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
movieId|userId|prediction|
+-------+------+----------+
 69| 11| 4.1210537|
 12| 11| 4.039393|
 94| 11| 3.4300377|
 30| 11| 3.2464716|
 27| 11| 3.0665927|
 10| 11| 2.8700979|
 64| 11| 2.7262347|
 11| 11| 2.0819232|
 45| 11| 1.9171729|
 13| 11| 1.3366563|
+-------+------+----------+